# Python'da Modelleme Kitaplıklarına Giriş

# 13.1 Pandas ve Model Kodu Arasında Arayüz
    Model geliştirme için ortak bir iş akışı, modelin kendisini oluşturmak için bir modelleme kitaplığına geçmeden önce veri yükleme ve temizleme için pandaların kullanılmasıdır. Model geliştirme sürecinin önemli bir parçası, makine öğreniminde özellik mühendisliği olarak adlandırılır. Bu, bir modelleme bağlamında yararlı olabilecek ham veri kümesinden bilgi çıkaran herhangi bir veri dönüşümünü veya analitiği açıklayabilir. Bu kitapta araştırdığımız veri toplama ve GroupBy araçları, genellikle bir yapı mühendisliği bağlamında kullanılmaktadır. "İyi" özellik mühendisliğinin detayları bu kitap için kapsam dışı olsa da, pandalarla veri manipülasyonu ve modellenmesi arasında mümkün olduğunca zahmetsiz geçiş yapmak için bazı yöntemler göstereceğim. Pandas ve diğer analiz kitaplıkları arasındaki temas noktası genellikle NumPy dizileridir. Bir DataFrame'i NumPy dizisine dönüştürmek için .values özelliğini kullanın.

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.DataFrame({
    'x0': [1, 2, 3, 4, 5],
    'x1': [0.01, -0.01, 0.25, -4.1, 0.],
    'y': [-1.5, 0., 3.6, 1.3, -2.]})

In [3]:
data

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [4]:
data.columns

Index(['x0', 'x1', 'y'], dtype='object')

In [5]:
data.values

array([[ 1.  ,  0.01, -1.5 ],
       [ 2.  , -0.01,  0.  ],
       [ 3.  ,  0.25,  3.6 ],
       [ 4.  , -4.1 ,  1.3 ],
       [ 5.  ,  0.  , -2.  ]])

    DataFrame'e geri dönüştürmek için, önceki bölümlerden hatırlayabileceğiniz gibi, isteğe bağlı sütun adlarıyla iki boyutlu bir ndarray geçirebilirsiniz.

In [6]:
df2 = pd.DataFrame(data.values, columns=['one', 'two', 'three'])

In [7]:
df2

,one,two,three
0,1.0,0.01,-1.5
1,2.0,-0.01,0.0
2,3.0,0.25,3.6
3,4.0,-4.10,1.3
4,5.0,0.00,-2.0


    .Values özniteliğinin, verileriniz homojen olduğunda (örneğin, tüm sayısal türler) kullanılması amaçlanmıştır. Heterojen verileriniz varsa, sonuç Python nesnelerinin bir ndarray'ı olacaktır.

In [8]:
df3 = data.copy()

In [9]:
df3['strings'] = ['a', 'b', 'c', 'd', 'e']

In [10]:
df3

,x0,x1,y,strings
0,1,0.01,-1.5,a
1,2,-0.01,0.0,b
2,3,0.25,3.6,c
3,4,-4.10,1.3,d
4,5,0.00,-2.0,e


In [11]:
df3.values

array([[1, 0.01, -1.5, 'a'],
       [2, -0.01, 0.0, 'b'],
       [3, 0.25, 3.6, 'c'],
       [4, -4.1, 1.3, 'd'],
       [5, 0.0, -2.0, 'e']], dtype=object)

    Bazı modellerde, sütunların yalnızca bir alt kümesini kullanmak isteyebilirsiniz. Değerlerle loc indekslemeyi kullanmanızı tavsiye edilir.

In [12]:
model_cols = ['x0', 'x1']

In [13]:
data.loc[:, model_cols].values

array([[ 1.  ,  0.01],
       [ 2.  , -0.01],
       [ 3.  ,  0.25],
       [ 4.  , -4.1 ],
       [ 5.  ,  0.  ]])

    Bazı kitaplıkların pandalar için yerel desteği vardır ve bu işlerin bir kısmını sizin için otomatik olarak yapar: DataFrame'den NumPy'ye dönüştürme ve model parametre adlarını çıktı tablolarının veya Serilerin sütunlarına ekleme. Diğer durumlarda, bu "meta veri yönetimini" manuel olarak yapmanız gerekecektir. Bölüm 12'de pandas Kategorik türüne ve pandas.get_dummies işlevine baktık. Örnek veri kümemizde sayısal olmayan bir sütunumuz olduğunu varsayalım:

In [14]:
data['category'] = pd.Categorical(['a', 'b', 'a', 'a', 'b'],
                                  categories=['a', 'b'])

In [15]:
data

,x0,x1,y,category
0,1,0.01,-1.5,a
1,2,-0.01,0.0,b
2,3,0.25,3.6,a
3,4,-4.10,1.3,a
4,5,0.00,-2.0,b


    'Kategori' sütununu kukla değişkenlerle değiştirmek istersek, kukla değişkenler oluşturur, 'kategori' sütununu kaldırır ve ardından sonucu birleştiririz.

In [16]:
dummies = pd.get_dummies(data.category, prefix='category')

In [17]:
data_with_dummies = data.drop('category', axis=1).join(dummies)

In [18]:
data_with_dummies

,x0,x1,y,category_a,category_b
0,1,0.01,-1.5,1,0
1,2,-0.01,0.0,0,1
2,3,0.25,3.6,1,0
3,4,-4.10,1.3,1,0
4,5,0.00,-2.0,0,1


    Bazı istatistiksel modelleri kukla değişkenlerle uydurmak için bazı nüanslar vardır. Basit sayısal sütunlardan daha fazlasına sahip olduğunuzda Patsy'yi (sonraki bölümün konusu) kullanmak daha basit ve daha az hata yapmaya meyilli olabilir.

# 13.2 Patsy ile Model Açıklamaları Oluşturma
    Patsy, R ve S istatistiksel programlama dilleri tarafından kullanılan formül sözdiziminden esinlenen (ancak tam olarak aynı olmayan) küçük dizge tabanlı "formül sözdizimi" ile istatistiksel modelleri (özellikle doğrusal modeller) açıklayan bir Python kitaplığıdır. Patsy, istatistik modellerinde doğrusal modelleri belirtmek için iyi bir şekilde desteklenmektedir, Patsy’nin formülleri şuna benzeyen özel bir dizgi sözdizimidir:
    y ~ x0 + x1
    A + b sözdizimi, a'yı b'ye eklemek anlamına gelmez, bunun yerine bunların model için oluşturulan tasarım matrisindeki terimler olduğu anlamına gelir. Patsy.dmatrices işlevi, bir veri kümesiyle birlikte (bir DataFrame veya bir dizi diktesi olabilir) bir formül dizesi alır ve doğrusal bir model için tasarım matrisleri üretir.

In [19]:
data = pd.DataFrame({
    'x0': [1, 2, 3, 4, 5],
    'x1': [0.01, -0.01, 0.25, -4.1, 0.],
    'y': [-1.5, 0., 3.6, 1.3, -2.]})

In [20]:
data

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [21]:
pip install patsy

Note: you may need to restart the kernel to use updated packages.


In [22]:
import patsy

In [23]:
y, X = patsy.dmatrices('y ~ x0 + x1', data)

In [24]:
y

DesignMatrix with shape (5, 1)
     y
  -1.5
   0.0
   3.6
   1.3
  -2.0
  Terms:
    'y' (column 0)

In [25]:
X

DesignMatrix with shape (5, 3)
  Intercept  x0     x1
          1   1   0.01
          1   2  -0.01
          1   3   0.25
          1   4  -4.10
          1   5   0.00
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'x1' (column 2)

    Bu Patsy DesignMatrix örnekleri, ek meta verilere sahip NumPy ndarraylerdir.

In [26]:
np.asarray(y)

array([[-1.5],
       [ 0. ],
       [ 3.6],
       [ 1.3],
       [-2. ]])

In [27]:
np.asarray(X)

array([[ 1.  ,  1.  ,  0.01],
       [ 1.  ,  2.  , -0.01],
       [ 1.  ,  3.  ,  0.25],
       [ 1.  ,  4.  , -4.1 ],
       [ 1.  ,  5.  ,  0.  ]])

    Intercept teriminin nereden geldiğini merak edebilirsiniz. Bu, sıradan en küçük kareler (OLS) regresyonu gibi doğrusal modeller için bir kuraldır. Modele + 0 terimini ekleyerek kesmeyi bastırabilirsiniz.

In [28]:
patsy.dmatrices('y ~ x0 + x1 + 0', data)[1]

DesignMatrix with shape (5, 2)
  x0     x1
   1   0.01
   2  -0.01
   3   0.25
   4  -4.10
   5   0.00
  Terms:
    'x0' (column 0)
    'x1' (column 1)

    Patsy nesneleri, sıradan bir en küçük kareler regresyonu gerçekleştiren numpy.linalg.lstsq gibi algoritmalara doğrudan aktarılabilir.

In [29]:
coef, resid, _, _ = np.linalg.lstsq(X, y)

/Users/veyseldogan/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  """Entry point for launching an IPython kernel.


    Model meta verileri design_info özniteliğinde tutulur, böylece bir Seri elde etmek için uydurulmuş katsayılara model sütun adları, örneğin:

In [30]:
coef

array([[ 0.31290976],
       [-0.07910564],
       [-0.26546384]])

In [31]:
coef = pd.Series(coef.squeeze(), index=X.design_info.column_names)

In [32]:
coef

Intercept    0.312910
x0          -0.079106
x1          -0.265464
dtype: float64

# Patsy Formüllerinde Veri Dönüşümleri
    Python kodunu Patsy formüllerinize karıştırabilirsiniz; Formülü değerlendirirken kütüphane, kapsama alanında kullandığınız fonksiyonları bulmaya çalışacaktır.

In [33]:
y, X = patsy.dmatrices('y ~ x0 + np.log(np.abs(x1) + 1)', data)

In [34]:
X

DesignMatrix with shape (5, 3)
  Intercept  x0  np.log(np.abs(x1) + 1)
          1   1                 0.00995
          1   2                 0.00995
          1   3                 0.22314
          1   4                 1.62924
          1   5                 0.00000
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'np.log(np.abs(x1) + 1)' (column 2)

    Yaygın olarak kullanılan bazı değişken dönüşümler arasında standartlaştırma (0 ve varyans 1 anlamına gelir) ve merkezleme (ortalamanın çıkarılması) bulunur. Patsy'nin bu amaç için yerleşik FONKSİYONLARI vardır:

In [35]:
y, X = patsy.dmatrices('y ~ standardize(x0) + center(x1)', data)

In [36]:
X

DesignMatrix with shape (5, 3)
  Intercept  standardize(x0)  center(x1)
          1         -1.41421        0.78
          1         -0.70711        0.76
          1          0.00000        1.02
          1          0.70711       -3.33
          1          1.41421        0.77
  Terms:
    'Intercept' (column 0)
    'standardize(x0)' (column 1)
    'center(x1)' (column 2)

    Modelleme sürecinin bir parçası olarak, bir modeli bir veri kümesine sığdırabilir, ardından modeli diğerine göre değerlendirebilirsiniz. Bu, daha sonra gözlemlenecek bir uzatma kısmı veya yeni veriler olabilir. Merkez ve standartlaştırma gibi dönüşümleri uygularken, modeli yeni verilere dayalı tahminler oluşturmak için kullanırken dikkatli olmalısınız. Yeni bir veri kümesini dönüştürürken orijinal veri kümesinin ortalama veya standart sapması gibi istatistikleri kullanmanız gerektiğinden bunlara durum bilgisi olan dönüştürmeler denir. Patsy.build_design_matrices işlevi, orijinal örnek içi veri kümesinden kaydedilen bilgileri kullanarak yeni örnek dışı verilere dönüşümler uygulayabilir.

In [37]:
new_data = pd.DataFrame({
    'x0': [6, 7, 8, 9],
    'x1': [3.1, -0.5, 0, 2.3],
    'y': [1, 2, 3, 4]})

In [38]:
new_X = patsy.build_design_matrices([X.design_info], new_data)

In [39]:
new_X

[DesignMatrix with shape (4, 3)
   Intercept  standardize(x0)  center(x1)
           1          2.12132        3.87
           1          2.82843        0.27
           1          3.53553        0.77
           1          4.24264        3.07
   Terms:
     'Intercept' (column 0)
     'standardize(x0)' (column 1)
     'center(x1)' (column 2)]

    Patsy formüllerinin bağlamındaki artı simgesi (+) toplama anlamına gelmediğinden, bir veri kümesinden ada göre sütun eklemek istediğinizde, bunları özel I işlevine sarmalısınız.

In [40]:
y, X = patsy.dmatrices('y ~ I(x0 + x1)', data)

In [41]:
X

DesignMatrix with shape (5, 2)
  Intercept  I(x0 + x1)
          1        1.01
          1        1.99
          1        3.25
          1       -0.10
          1        5.00
  Terms:
    'Intercept' (column 0)
    'I(x0 + x1)' (column 1)

# Kategorik Veriler ve Patsy
    Sayısal olmayan veriler, bir model tasarım matrisi için birçok farklı yolla dönüştürülebilir. Bu konunun tam olarak ele alınması bu kitabın kapsamı dışındadır ve en iyi istatistik dersi ile birlikte incelenebilir. Bir Patsy formülünde sayısal olmayan terimler kullandığınızda, bunlar varsayılan olarak kukla değişkenlere dönüştürülür. Bir kesişme varsa, eşdoğrusallığı önlemek için seviyelerden biri dışarıda bırakılacaktır.

In [42]:
data = pd.DataFrame({
    'key1': ['a', 'a', 'b', 'b', 'a', 'b', 'a', 'b'],
    'key2': [0, 1, 0, 1, 0, 1, 0, 0],
    'v1':[1,2,3,4,5,6,7,8],
    'v2': [-1, 0, 2.5, -0.5, 4.0, -1.2, 0.2, -1.7]
})

In [43]:
y, X = patsy.dmatrices('v2 ~ key1', data)

In [44]:
X

DesignMatrix with shape (8, 2)
  Intercept  key1[T.b]
          1          0
          1          0
          1          1
          1          1
          1          0
          1          1
          1          0
          1          1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)

    Modelden kesmeyi çıkarırsanız, her bir kategori değeri için sütunlar model tasarım matrisine dahil edilecektir.

In [45]:
y, X = patsy.dmatrices('v2 ~ key1 + 0', data)

In [46]:
X

DesignMatrix with shape (8, 2)
  key1[a]  key1[b]
        1        0
        1        0
        0        1
        0        1
        1        0
        0        1
        1        0
        0        1
  Terms:
    'key1' (columns 0:2)

    Sayısal sütunlar, C işlevi ile kategorik olarak yorumlanabilir.

In [47]:
y, X = patsy.dmatrices('v2 ~ C(key2)', data)

In [48]:
X

DesignMatrix with shape (8, 2)
  Intercept  C(key2)[T.1]
          1             0
          1             1
          1             0
          1             1
          1             0
          1             1
          1             0
          1             0
  Terms:
    'Intercept' (column 0)
    'C(key2)' (column 1)

    Bir modelde birden fazla kategorik terim kullandığınızda, örneğin varyans analizi (ANOVA) modellerinde kullanılabilen key1: key2 formunun etkileşim terimlerini dahil edebileceğiniz için işler daha karmaşık hale gelebilir.

In [49]:
data['key2'] = data['key2'].map({0: 'zero', 1: 'one'})

In [50]:
data

,key1,key2,v1,v2
0,a,zero,1,-1.0
1,a,one,2,0.0
2,b,zero,3,2.5
3,b,one,4,-0.5
4,a,zero,5,4.0
5,b,one,6,-1.2
6,a,zero,7,0.2
7,b,zero,8,-1.7


In [51]:
y, X = patsy.dmatrices('v2 ~ key1 + key2', data)

In [52]:
X

DesignMatrix with shape (8, 3)
  Intercept  key1[T.b]  key2[T.zero]
          1          0             1
          1          0             0
          1          1             1
          1          1             0
          1          0             1
          1          1             0
          1          0             1
          1          1             1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)
    'key2' (column 2)

In [53]:
y, X = patsy.dmatrices('v2 ~ key1 + key2 + key1:key2', data)

In [54]:
X

DesignMatrix with shape (8, 4)
  Intercept  key1[T.b]  key2[T.zero]  key1[T.b]:key2[T.zero]
          1          0             1                       0
          1          0             0                       0
          1          1             1                       1
          1          1             0                       0
          1          0             1                       0
          1          1             0                       0
          1          0             1                       0
          1          1             1                       1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)
    'key2' (column 2)
    'key1:key2' (column 3)

# 13.3 İstatistik modellerine giriş
    statsmodels, birçok türden istatistiksel modeli uydurmak, istatistiksel testleri gerçekleştirmek ve veri keşfi ve görselleştirmek için bir Python kitaplığıdır. Statsmodels daha “klasik” sıklıkçı istatistiksel yöntemler içerirken, Bayes yöntemleri ve makine öğrenme modelleri diğer kütüphanelerde bulunur.
    İstatistik modellerinde bulunan bazı model türleri şunları içerir:
    • Doğrusal modeller, genelleştirilmiş doğrusal modeller ve sağlam doğrusal modeller
    • Doğrusal karışık efekt modelleri
    • Varyans analizi (ANOVA) yöntemleri
    • Zaman serisi süreçleri ve durum uzayı modelleri
    • Genelleştirilmiş anlar yöntemi

# Doğrusal Modellerin Tahmin Edilmesi
    İstatistik modellerinde, daha basitten (örneğin, sıradan en küçük kareler) daha karmaşığa (örneğin, yinelemeli olarak yeniden ağırlıklandırılmış en küçük kareler) çeşitli doğrusal regresyon modeli vardır. İstatistik modellerinde doğrusal modeller iki farklı ana arayüze sahiptir: dizi tabanlı ve formül tabanlı. Bunlara, şu API modülü içe aktarmalarıyla erişilir:

In [55]:
pip install statsmodels 

Note: you may need to restart the kernel to use updated packages.


In [57]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

    Bunların nasıl kullanılacağını göstermek için bazı rastgele verilerden doğrusal bir model oluşturuyoruz.

In [58]:
def dnorm(mean, variance, size=1):
    if isinstance(size, int):
        size = size,
        return mean + np.sqrt(variance) * np.random.randn(*size)

In [59]:
np.random.seed(12345)

In [62]:
N=100
X = np.c_[dnorm(0, 0.4, size=N),
          dnorm(0, 0.6, size=N),
          dnorm(0, 0.2, size=N)]
eps = dnorm(0, 0.1, size=N)
beta = [0.1, 0.3, 0.5]
y = np.dot(X, beta) + eps

    Burada, beta parametreleri bilinen "gerçek" modeli yazdık. Bu durumda, dnorm, belirli bir ortalama ve varyans ile normal olarak dağıtılmış veriler oluşturmak için yardımcı bir işlevdir. Şimdi elimizde:

In [63]:
X[:5]

array([[-0.12946849, -1.21275292,  0.50422488],
       [ 0.30291036, -0.43574176, -0.25417986],
       [-0.32852189, -0.02530153,  0.13835097],
       [-0.35147471, -0.71960511, -0.25821463],
       [ 1.2432688 , -0.37379916, -0.52262905]])

In [64]:
y[:5]

array([ 0.42786349, -0.67348041, -0.09087764, -0.48949442, -0.12894109])

    Doğrusal bir model genellikle daha önce Patsy ile gördüğümüz gibi bir kesişme terimi ile donatılmıştır. Sm.add_constant işlevi, mevcut bir matrise bir kesme sütunu ekleyebilir.

In [65]:
X_model = sm.add_constant(X)

In [66]:
X_model[:5]

array([[ 1.        , -0.12946849, -1.21275292,  0.50422488],
       [ 1.        ,  0.30291036, -0.43574176, -0.25417986],
       [ 1.        , -0.32852189, -0.02530153,  0.13835097],
       [ 1.        , -0.35147471, -0.71960511, -0.25821463],
       [ 1.        ,  1.2432688 , -0.37379916, -0.52262905]])

    Sm.OLS sınıfı, sıradan bir en küçük kareler doğrusal regresyona uyabilir.

In [67]:
model = sm.OLS(y, X)

    Modelin uyum yöntemi, tahmini model parametrelerini ve diğer teşhisleri içeren bir regresyon sonuçları nesnesi döndürür.

In [68]:
results = model.fit()

In [69]:
results.params

array([0.17826108, 0.22303962, 0.50095093])

    Sonuçların özet yöntemi, sonuçların tanılama çıktısını detaylandıran bir modeli yazdırabilir.
model:

In [70]:
print(results.summary())

                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.430
Model:                            OLS   Adj. R-squared (uncentered):              0.413
Method:                 Least Squares   F-statistic:                              24.42
Date:                Sun, 27 Dec 2020   Prob (F-statistic):                    7.44e-12
Time:                        01:27:52   Log-Likelihood:                         -34.305
No. Observations:                 100   AIC:                                      74.61
Df Residuals:                      97   BIC:                                      82.42
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

    Buradaki parametre adlarına x1, x2 vb. Genel adlar verilmiştir. Bunun yerine, tüm model parametrelerinin bir DataFrame'de olduğunu varsayalım.

In [71]:
data = pd.DataFrame(X, columns=['col0', 'col1', 'col2'])

In [72]:
data['y'] = y

In [73]:
data[:5]

,col0,col1,col2,y
0,-0.129468,-1.212753,0.504225,0.427863
1,0.302910,-0.435742,-0.254180,-0.673480
2,-0.328522,-0.025302,0.138351,-0.090878
3,-0.351475,-0.719605,-0.258215,-0.489494
4,1.243269,-0.373799,-0.522629,-0.128941


    Şimdi istatistik modelleri formül API'sini ve Patsy formül dizelerini kullanabiliriz.

In [74]:
results = smf.ols('y ~ col0 + col1 + col2', data=data).fit()

In [75]:
results.params

Intercept    0.033559
col0         0.176149
col1         0.224826
col2         0.514808
dtype: float64

In [76]:
results.tvalues

Intercept    0.952188
col0         3.319754
col1         4.850730
col2         6.303971
dtype: float64

    Statisticsmodels'in sonuçları DataFrame sütun adları eklenmiş olarak Seriler olarak nasıl döndürdüğünü gözlemleyin. Ayrıca formülleri ve pandas nesnelerini kullanırken add_constant kullanmamıza gerek yoktur.

    Yeni örneklem dışı veriler göz önüne alındığında, tahmini model parametreleri verildiğinde tahmin edilen değerleri hesaplayabilirsiniz.

In [77]:
results.predict(data[:5])

0   -0.002327
1   -0.141904
2    0.041226
3   -0.323070
4   -0.100535
dtype: float64

    Doğrusal model sonuçlarının analizi, teşhisi ve keşfedebileceğiniz istatistik modellerinde görselleştirilmesi için birçok ek araç vardır. Sıradan en küçük karelerin ötesinde başka türde doğrusal modeller de vardır.

# Zaman Serisi Süreçlerinin Tahmin Edilmesi
    İstatistik modellerindeki diğer bir model sınıfı, zaman serisi analizi içindir. Bunlar arasında otoregresif süreçler, Kalman filtreleme ve diğer durum uzayı modelleri ve çok değişkenli otoregresif modeller vardır. Otoregresif bir yapı ve gürültü ile bazı zaman serisi verilerini simüle edelim.

In [78]:
init_x = 4

In [79]:
import random
values = [init_x, init_x]
N=1000

In [80]:
b0=0.8
b1 = -0.4
noise = dnorm(0, 0.1, N)
for i in range(N):
    new_x = values[-1] * b0 + values[-2] * b1 + noise[i]
    values.append(new_x)

    Bu veri 0.8 ve –0.4 parametreli bir AR (2) yapısına (iki gecikme) sahiptir. Bir AR modeline uyduğunuzda, eklenecek gecikmeli terimlerin sayısını bilmeyebilirsiniz, bu nedenle modeli daha fazla sayıda gecikmeyle sığdırabilirsiniz.

In [81]:
MAXLAGS = 5

In [82]:
model = sm.tsa.AR(values)

/Users/veyseldogan/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/statsmodels/tsa/ar_model.py:791: FutureWarning: 
statsmodels.tsa.AR has been deprecated in favor of statsmodels.tsa.AutoReg and
statsmodels.tsa.SARIMAX.

AutoReg adds the ability to specify exogenous variables, include time trends,
and add seasonal dummies. The AutoReg API differs from AR since the model is
treated as immutable, and so the entire specification including the lag
length must be specified when creating the model. This change is too
substantial to incorporate into the existing AR api. The function
ar_select_order performs lag length selection for AutoReg models.

AutoReg only estimates parameters using conditional MLE (OLS). Use SARIMAX to
estimate ARX and related models using full MLE via the Kalman Filter.

To silence this warning and continue using AR until it is removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.ar_model.AR', FutureWarning)

  warnings.warn(AR_DE

In [83]:
results = model.fit(MAXLAGS)

    Sonuçlarda tahmin edilen parametrelerde önce kesişme, ardından ilk iki gecikme için tahminler bulunur.

In [84]:
results.params

array([-0.00616093,  0.78446347, -0.40847891, -0.01364148,  0.01496872,
        0.01429462])

# 13.4 Scikit-learn'e giriş
    scikit-learn, en yaygın kullanılan ve güvenilir genel amaçlı Python makine öğrenimi araç setlerinden biridir. Model seçimi ve değerlendirmesi, veri dönüşümü, veri yükleme ve model kalıcılığı için araçlar içeren çok çeşitli standart denetimli ve denetimsiz makine öğrenimi yöntemleri içerir. Bu modeller sınıflandırma, kümeleme, tahmin ve diğer ortak görevler için kullanılabilir. Bu bölümde scikit-learn API stilinin kısa bir çeşidini vereceğiz.

    Örnek olarak, 1912'de batan Titanik'teki yolcu hayatta kalma oranları hakkında bir Kaggle yarışmasından klasik bir veri kümesini kullanıyorum. Test ve eğitim veri kümesini pandaları kullanarak yüklüyoruz.

In [85]:
train = pd.read_csv('datasets/titanic/train.csv')

In [86]:
test = pd.read_csv('datasets/titanic/test.csv')

In [87]:
train[:4]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S


    İstatistik modelleri ve scikit-learn gibi kitaplıklar genellikle eksik verilerle beslenemez, bu nedenle eksik verileri içeren herhangi bir veri olup olmadığını görmek için sütunlara bakarız.

In [88]:
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [89]:
test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

    Bunun gibi istatistiklerde ve makine öğrenimi örneklerinde, tipik bir görev, verilerdeki özelliklere dayalı olarak bir yolcunun hayatta kalıp kalmayacağını tahmin etmektir. Bir model, bir eğitim veri kümesine yerleştirilir ve ardından örneklem dışı bir test veri kümesinde değerlendirilir.

    Yaşı bir tahmin aracı olarak kullanmak istiyoruz, ancak eksik verileri var. Eksik veri atamasını yapmanın birkaç yolu vardır, ancak basit bir tane yapacağız ve her iki tablodaki boş değerleri doldurmak için eğitim veri kümesinin medyanını kullanacağız.

In [90]:
impute_value = train['Age'].median()

In [91]:
train['Age'] = train['Age'].fillna(impute_value)

In [92]:
test['Age'] = test['Age'].fillna(impute_value)

    Şimdi modellerimizi belirlememiz gerekiyor. 'Sex' sütununun kodlanmış versiyonu olarak bir IsFemale sütunu ekliyoruz.

In [93]:
train['IsFemale'] = (train['Sex'] == 'female').astype(int)

In [94]:
test['IsFemale'] = (test['Sex'] == 'female').astype(int)

    Sonra bazı model değişkenlerine karar veriyoruz ve NumPy dizileri oluşturuyoruz.

In [95]:
predictors = ['Pclass', 'IsFemale', 'Age']

In [96]:
X_train = train[predictors].values

In [97]:
X_test = test[predictors].values

In [99]:
y_train = train['Survived'].values

In [100]:
X_train[:5]

array([[ 3.,  0., 22.],
       [ 1.,  1., 38.],
       [ 3.,  1., 26.],
       [ 1.,  1., 35.],
       [ 3.,  0., 35.]])

In [101]:
y_train[:5]

array([0, 1, 1, 1, 0])

    Scikit-learn'ün LogisticRegression modelini kullanıyoruz ve bir model örneği oluşturuyoruz.

In [102]:
from sklearn.linear_model import LogisticRegression

In [103]:
model = LogisticRegression()

    İstatistik modellerine benzer şekilde, bu modeli modelin uyum yöntemini kullanarak eğitim verilerine uydurabiliriz.

In [104]:
model.fit(X_train, y_train)

LogisticRegression()

    Şimdi, model.predict'i kullanarak test veri kümesi için tahminler oluşturabiliriz.

In [106]:
y_predict = model.predict(X_test)

In [107]:
y_predict[:10]

array([0, 0, 0, 0, 1, 0, 1, 0, 1, 0])

    Test veri kümesi için doğru değerlere sahipseniz, bir doğruluk yüzdesi veya başka bir hata ölçüsü hesaplayabilirsiniz.
    (y_true == y_predict).mean()

    Uygulamada, model eğitiminde genellikle birçok ek karmaşıklık katmanı vardır. Birçok modelin ayarlanabilen parametreleri vardır ve eğitim verilerine aşırı uymayı önlemek için parametre ayarlamada kullanılabilecek çapraz doğrulama gibi teknikler vardır. Bu, genellikle yeni verilerde daha iyi tahmin performansı veya sağlamlık sağlayabilir. Çapraz doğrulama, örneklem dışı öngörü simüle etmek için eğitim verilerini bölerek çalışır. Ortalama hata karesi gibi bir model doğruluk puanına dayalı olarak, model parametreleri üzerinde bir ızgara araması gerçekleştirilebilir. Lojistik regresyon gibi bazı modeller, yerleşik çapraz doğrulamaya sahip tahminci sınıflarına sahiptir. Örneğin, LogisticRegressionCV sınıfı, C model düzenleme parametresinde ne kadar ayrıntılı bir ızgara aramasının yapılacağını gösteren bir parametre ile kullanılabilir.

In [109]:
from sklearn.linear_model import LogisticRegressionCV

In [111]:
model_cv = LogisticRegressionCV(10)

In [112]:
model_cv.fit(X_train, y_train)

LogisticRegressionCV()

    Elle çapraz doğrulama yapmak için, veri bölme sürecini işleyen cross_val_score yardımcı işlevini kullanabilirsiniz. Örneğin, modelimizi eğitim verilerinin örtüşmeyen dört bölümüyle çapraz doğrulamak için şunları yapabiliriz:

In [114]:
from sklearn.model_selection import cross_val_score

In [115]:
model = LogisticRegression(C=10)

In [116]:
scores = cross_val_score(model, X_train, y_train, cv=4)

In [117]:
scores

array([0.77578475, 0.79820628, 0.77578475, 0.78828829])

    Varsayılan puanlama ölçüsü modele bağlıdır, ancak açık bir puanlama işlevi seçmek mümkündür. Çapraz doğrulanmış modellerin eğitilmesi daha uzun sürer, ancak genellikle daha iyi model performansı sağlayabilir.

# 13.5 Eğitiminize Devam Etme
    Python'da veya bir Python kullanıcı arayüzüyle uygulanan çeşitli istatistik ve makine öğrenimi türleri için gittikçe daha fazla çerçeve var. Modelleme ve veri bilimi araçlarına adanmış başka birçok kitap da vardır. Bazı mükemmel olanlar:
    • Andreas Mueller ve Sarah Guido (O’Reilly) tarafından Python ile Makine Öğrenimine Giriş
    • Python Veri Bilimi El Kitabı, Jake VanderPlas (O’Reilly)
    • Sıfırdan Veri Bilimi: Python ile İlk İlkeler, Joel Grus (O’Reilly)
    • Sebastian Raschka (Packt Publishing) tarafından Python Makine Öğrenimi
    • Aurélien Géron (O’Reilly) tarafından Scikit-Learn ve TensorFlow ile Uygulamalı Makine Öğrenimi